# surplus 2.x.y playground notebook

wrangling with environments for devbox users using codium/vs code:

```text
$ devbox shell                    # enter devbox env
(surplus-py3.11) (devbox) $ exit  # leave poetry env
(devbox) $ codium .               # open ide
```

In [1]:
# converting nominatim keys to OUTPUT_LINE_X_KEYS format

keys = """
region, county, state, state_district, country, continent
"""

split_keys = [f'"{key.strip()}"' for key in keys.strip().split(",")]

print(f"OUTPUT_LINE_X_KEYS: Final[tuple[str, ...]] = ({','.join(split_keys)},)")

OUTPUT_LINE_X_KEYS: Final[tuple[str, ...]] = ("region","county","state","state_district","country","continent",)


In [1]:
from surplus import PlusCodeQuery, LocalCodeQuery, LatlongQuery, StringQuery
from surplus import Latlong, Result
from surplus import SurplusDefaultGeocoding

geocoding = SurplusDefaultGeocoding()

## Generic Result NamedTuple

In [3]:
nom_result = Result[int](3)

try:
    1 / 0
except Exception as exc:
    exc_result = Result[int](-1, error=exc)

print("{}\t{:<40}\t{}".format(bool(nom_result), repr(nom_result.error), nom_result.get()))
print(
    "{}\t{:<40}\t{}".format(
        bool(exc_result), repr(exc_result.error), exc_result.cry(string=True)
    )
)
print("{}\t{:<40}\t{}".format(bool(exc_result), repr(exc_result.error), exc_result.get()))

True	None                                    	3
False	ZeroDivisionError('division by zero')   	division by zero (ZeroDivisionError)


ZeroDivisionError: division by zero

## Query Types

In [ ]:
PlusCodeQuery(code="6PH58QMF+FV").to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [ ]:
plus_code = LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_full_plus_code(
    geocoder=geocoding.geocoder
)

PlusCodeQuery(code=plus_code.get()).to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [ ]:
LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_lat_long_coord(
    geocoder=geocoding.geocoder
)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [ ]:
LatlongQuery(
    latlong=Latlong(latitude=1.33318835, longitude=103.77461234638255)
).to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.33318835, longitude=103.77461234638255), error=None)

In [ ]:
StringQuery(query="Ngee Ann Polytechnic").to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.33318835, longitude=103.77461234638255), error=None)

## return dictionary of `reverser` function

all the necessary keys (see `SHAREABLE_TEXT_LINE_*` constants) should be at the top-level of the dictionary.
these keys will be casted into strings for safety guarantee when shareable text lines are being generated.

while not necessary, consider keeping the original response dict under the "raw" key.
helps with debugging using `-d/--debug`!

In [ ]:
reverser_return = {
    "amenity": "Ngee Ann Polytechnic",
    "house_number": "535",
    "road": "Clementi Road",
    "suburb": "Bukit Timah",
    "city": "Singapore",
    "county": "Northwest",
    "ISO3166-2-lvl6": "SG-03",
    "postcode": "599489",
    "country": "Singapore",
    "country_code": "sg",
    "raw": {
        "place_id": 297946059,
        "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
        "osm_type": "relation",
        "osm_id": 2535118,
        "lat": "1.33318835",
        "lon": "103.77461234638255",
        "class": "amenity",
        "type": "university",
        "place_rank": 30,
        "importance": 0.34662169301918117,
        "addresstype": "amenity",
        "name": "Ngee Ann Polytechnic",
        "display_name": "Ngee Ann Polytechnic, 535, Clementi Road, Bukit Timah, Singapore, Northwest, 599489, Singapore",
        "address": {
            "amenity": "Ngee Ann Polytechnic",
            "house_number": "535",
            "road": "Clementi Road",
            "suburb": "Bukit Timah",
            "city": "Singapore",
            "county": "Northwest",
            "ISO3166-2-lvl6": "SG-03",
            "postcode": "599489",
            "country": "Singapore",
            "country_code": "sg",
        },
        "boundingbox": ["1.3289692", "1.3372184", "103.7701481", "103.7783945"],
    },
    "latitude": 1.33318835,
    "longitude": 103.77461234638255,
}

In [ ]:
import pprint

latlong = LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_lat_long_coord(
    geocoder=geocoding.geocoder
)
if not latlong:
    latlong.cry()

else:
    location = geocoding.reverser(latlong.get())
    pprint.pprint(location)

{'ISO3166-2-lvl6': 'SG-03',
 'amenity': 'Ngee Ann Polytechnic',
 'city': 'Singapore',
 'country': 'Singapore',
 'country_code': 'sg',
 'county': 'Northwest',
 'house_number': '535',
 'latitude': 1.33318835,
 'longitude': 103.77461234638255,
 'neighbourhood': 'Ewart Park',
 'postcode': '599489',
 'raw': {'address': {'ISO3166-2-lvl6': 'SG-03',
                     'amenity': 'Ngee Ann Polytechnic',
                     'city': 'Singapore',
                     'country': 'Singapore',
                     'country_code': 'sg',
                     'county': 'Northwest',
                     'house_number': '535',
                     'neighbourhood': 'Ewart Park',
                     'postcode': '599489',
                     'road': 'Clementi Road',
                     'suburb': 'Bukit Timah'},
         'addresstype': 'amenity',
         'boundingbox': ['1.3289692',
                         '1.3372184',
                         '103.7701481',
                         '103.7783945'],
  

## 2.1.0: adventures in of shortening global/full Plus Codes

### testing rate-limited and cached default geocoding functions

In [2]:
test_geocoding = SurplusDefaultGeocoding(user_agent="surplus/playground")

In [2]:
from timeit import timeit


test_stmt = """\
print(1)
test_geocoding.geocoder("Wisma Atria")                                  # instant
print(2)
test_geocoding.geocoder("Temasek Polytechnic")                          # after 1 second
print(3)
location = test_geocoding.geocoder("Ngee Ann Polytechnic")              # after 1 second
print(4)
test_geocoding.reverser(f"{location.latitude}, {location.longitude}")   # instant
print(5)
test_geocoding.reverser(f"{location.latitude}, {location.longitude}")   # instant (cached)
print()
"""

time_cold_call = timeit(test_stmt, globals=globals(), number=1)  # expecting 3-4 seconds
time_2nd_call = timeit(test_stmt, globals=globals(), number=1)  # should be instant

print(
    f"{time_cold_call:.10f}s\t->\t{time_2nd_call:.10f}s\t\t({time_2nd_call - time_cold_call}s)"
)

1
2
3
4
5

1
2
3
4
5

3.1107698050s	->	0.0000886890s		(-3.1106811160002508s)


### reversing the query latlong and using the address information to form a locality

In [3]:
level = 13

In [4]:
(
    au_response := geocoding.reverser(
        (
            au_target := (
                LocalCodeQuery(
                    "G227+XF", "St Lucia, Queensland, Australia"
                ).to_lat_long_coord(geocoding.geocoder)
            )
        ).get(),
        level=level,
    )
)

au_locality = f"{au_response['suburb']}, {au_response['city_district']}, {au_response['state']}, {au_response['country']}"
print(au_locality)

(
    us_response := geocoding.reverser(
        (
            us_target := (
                LocalCodeQuery("77Q4+7X", "Austin, Texas, USA").to_lat_long_coord(
                    geocoding.geocoder
                )
            )
        ).get(),
        level=level,
    )
)

us_locality = f"{us_response['city']}, {us_response['county']}, {us_response['state']}, {us_response['country']}"
print(us_locality)

St Lucia, St Lucia, Queensland, Australia
Austin, Travis County, Texas, United States


### getting boundary boxes

In [13]:
from geopy.geocoders import Nominatim
from pprint import pprint

target_query: Result[Latlong] = au_target
target_locality: str = au_locality

raw_geocoding = Nominatim(user_agent="surplus/playground")
latlong = raw_geocoding.geocode(target_locality)
pprint(latlong.raw)
print()

# done: now implmented in surplus as surplus.Latlong.bounding_box
locality_latlong = geocoding.geocoder(target_locality)
pprint(locality_latlong)

{'addresstype': 'suburb',
 'boundingbox': ['-27.5187362', '-27.4787362', '152.9881642', '153.0281642'],
 'class': 'place',
 'display_name': 'St Lucia, Brisbane City, Queensland, 4072, Australia',
 'importance': 0.27501,
 'lat': '-27.4987362',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'http://osm.org/copyright',
 'lon': '153.0081642',
 'name': 'St Lucia',
 'osm_id': 88800268,
 'osm_type': 'node',
 'place_id': 54477898,
 'place_rank': 19,
 'type': 'suburb'}

Latlong(latitude=-27.4987362, longitude=153.0081642, bounding_box=[-27.5187362, -27.4787362, 152.9881642, 153.0281642])


In [15]:
# based on <https://github.com/google/open-location-code/wiki/Guidance-for-shortening-codes>

target_latlong = target_query.get()
if locality_latlong.bounding_box is None:
    ...  # raise some error

print(locality_latlong.bounding_box)
check1 = (
    # The center point of the feature is within 0.4 degrees latitude and 0.4 degrees longitude
    (
        (target_latlong.latitude - 0.4)
        <= locality_latlong.latitude
        <= (target_latlong.latitude + 0.4)
    ),
    (
        (target_latlong.longitude - 0.4)
        <= locality_latlong.longitude
        <= (target_latlong.longitude + 0.4)
    ),
    # The bounding box of the feature is less than 0.8 degrees high and wide.
    abs(locality_latlong.bounding_box[0] - locality_latlong.bounding_box[1]) < 0.8,
    abs(locality_latlong.bounding_box[2] - locality_latlong.bounding_box[3]) < 0.8,
)


check2 = (
    # The center point of the feature is within 0.4 degrees latitude and 0.4 degrees longitude
    (
        (target_latlong.latitude - 8)
        <= locality_latlong.latitude
        <= (target_latlong.latitude + 8)
    ),
    (
        (target_latlong.longitude - 8)
        <= locality_latlong.longitude
        <= (target_latlong.longitude + 8)
    ),
    # The bounding box of the feature is less than 0.8 degrees high and wide.
    abs(locality_latlong.bounding_box[0] - locality_latlong.bounding_box[1]) < 16,
    abs(locality_latlong.bounding_box[2] - locality_latlong.bounding_box[3]) < 16,
)

print(check1)
print(check2)

[-27.5187362, -27.4787362, 152.9881642, 153.0281642]
(True, True, True, True)
(True, True, True, True)


In [16]:
from pluscodes import encode

target_plus_code = encode(
    lat=target_latlong.latitude, lon=target_latlong.longitude, code_length=10
)
portion_plus_code = ""

if check1:
    portion_plus_code = target_plus_code[4:]
    print(portion_plus_code, target_locality)

elif check2:
    portion_plus_code = target_plus_code[2:]
    print(portion_plus_code, target_locality)

else:
    print(
        "info: could not determine a suitable geographical feature to use as locality for shortening."
    )
    print(plus_code)

G227+XF St Lucia, St Lucia, Queensland, Australia


## machine fingerprinting attempt

because of nominatim's acceptable usage policy  
<https://operations.osmfoundation.org/policies/nominatim/>

In [ ]:
from hashlib import shake_256 as _hashlib_shake_256
from platform import platform as _platform_platform
from socket import gethostname as _socket_gethostname
from uuid import getnode as _uuid_getnode
from surplus import VERSION, VERSION_SUFFIX


def generate_fingerprinted_user_agent() -> Result[str]:
    """
    function that attempts to return a unique user agent string.

    returns Result[str]
        this result will always have a valid value as erroneous results will have a
        resulting value of 'surplus/<version>/generic-user'
        valid results will have a value of 'surplus/<version>/<fingerprint>', where
        fingerprint is a 12 character hexadecimal string
    """
    version: str = ".".join([str(v) for v in VERSION]) + VERSION_SUFFIX

    try:
        system_info: str = _platform_platform()
        hostname: str = _socket_gethostname()
        mac_address: str = ":".join(
            [
                "{:02x}".format((_uuid_getnode() >> elements) & 0xFF)
                for elements in range(0, 2 * 6, 2)
            ][::-1]
        )
        unique_info: str = f"{version}-{system_info}-{hostname}-{mac_address}"

        print(f"{version=}")
        print(f"{system_info=}")
        print(f"{hostname=}")
        print(f"{mac_address=}")

    except Exception as exc:
        return Result[str](f"surplus/{version} (generic-user)", error=exc)

    fingerprint: str = _hashlib_shake_256(unique_info.encode()).hexdigest(5)

    return Result[str](f"surplus/{version} ({fingerprint})")